# Train and save text classification model

### Import modules

In [1]:
import src.data.make_dataset as make_dataset
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [6]:
train_data, valid_data, classes = make_dataset.load_processed("../data/processed/")

In [2]:
train_data, valid_data, classes = make_dataset.load_data("../data/raw/")

INFO:root:Number of training samples:    3543746
INFO:root:Number of test     samples:     151860
INFO:root:Completed data shuffling.
INFO:root:Completed categories list.


In [7]:
train_data[0][15431]

'Hypocalcimia after thydroidectomy - Want new management?'

In [8]:
import src.models.train_model as train_model


In [ ]:
train_model.train()